#**Chatbot Training and Response Generation with BART**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/

Mounted at /content/drive


In [3]:
# # !pip install transformers[torch] vncorenlp
# !pip install datasets

In [4]:
# pip install transformers --upgrade

## 1. Importing Libraries and Setting Up the Environment

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BartphoTokenizer, BartForConditionalGeneration, AdamW
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import logging
import pandas as pd
from sklearn.decomposition import PCA
import tensorflow as tf
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer

#import for cbow
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Setup logging
logging.basicConfig(level=logging.INFO)

In [7]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# device = torch.device("cpu")

## 2. Loading Pretrained BART Model and Tokenizer

In [9]:
# Load BARTpho tokenizer and model
model_name = 'vinai/bartpho-syllable'

try:
    # Use BartphoTokenizer for BARTpho models
    tokenizer = BartphoTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    raise e

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

You are using a model of type mbart to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


## 3. Defining Hyperparameters

In [10]:
# Hyperparameters
batch_size = 16
gradient_accumulation_steps = 4
learning_rate = 5e-5
num_epochs = 3
warmup_steps = 100
max_length = 1024

## 4. Dataset for Demonstration

In [11]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('/content/drive/MyDrive/ChatBox/data/processed_medical.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# Kiểm tra dữ liệu
df.head()




,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng niêm_mạc,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng đường h...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng đường t...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng đường_s...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...


In [12]:
df.shape

(13693, 3)

# *Data Preparation*

In [13]:
# # Hàm để thay thế dấu '_' bằng khoảng trắng
# def replace_underscores(text_list):
#     return [text.replace('_', ' ') for text in text_list]

# Data Preparation
titles = df['Title'].tolist()
detailed_contents = df['Detailed Content'].tolist()

# Đảm bảo tất cả các phần tử trong danh sách là chuỗi
titles = [str(title) for title in titles]
detailed_contents = [str(content) for content in detailed_contents]

# # Thay thế dấu '_' bằng khoảng trắng
# titles = replace_underscores(titles)
# detailed_contents = replace_underscores(detailed_contents)

# Kiểm tra dữ liệu sau khi thay thế
print(titles[:5])  # In ra 5 tiêu đề đầu tiên để kiểm tra
print(detailed_contents[:5])  # In ra 5 nội dung chi tiết đầu tiên để kiểm tra

['rào_cản tự_nhiên chống lại nhiễm_trùng da', 'rào_cản tự_nhiên chống lại nhiễm_trùng niêm_mạc', 'rào_cản tự_nhiên chống lại nhiễm_trùng đường hô_hấp', 'rào_cản tự_nhiên chống lại nhiễm_trùng đường tiêu_hóa', 'rào_cản tự_nhiên chống lại nhiễm_trùng đường_sinh_dục tiết_niệu']
['da thường chặn các vi_sinh_vật xâm_nhập trừ khi nó bị tổn_thương ví_dụ do động_vật chân_đốt chấn_thương ống thông iv hoặc phẫu_thuật rạch các ngoại_lệ bao_gồm virus papilloma_người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides_stercoralis những loại gây bệnh_nhiễm giun_móc virus papilloma người có_thể xâm_nhập vào da bình_thường gây ra mụn cóc một_số ký_sinh_trùng ví_dụ strongyloides stercoralis những loại gây bệnh nhiễmgiun móc', 'nhiều màng_nhầy ngập trong chất tiết có đặc_tính kháng khuẩn ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệt và nước_mắt có chứa lysozyme chất này chia_cắt liên_kết axit muramic trong thành tế_bào vi_khuẩn đặc_biệt là ở các sinh_vật gram dươ

In [14]:
import re

# Hàm tìm từ ghép có dấu '_'
def find_compound_words(texts):
    compound_words = set()
    for text in texts:
        # Tìm các từ chứa dấu '_'
        matches = re.findall(r'\b\w+_\w+\b', text)
        compound_words.update(matches)
    return list(compound_words)

# Tìm từ ghép trong titles và detailed_contents
compound_words_titles = find_compound_words(titles)
compound_words_contents = find_compound_words(detailed_contents)

# Kết hợp từ ghép từ cả titles và detailed_contents và đảm bảo tính duy nhất
compound_words = list(set(compound_words_titles + compound_words_contents))

# Kiểm tra số lượng từ ghép đã tìm thấy và các từ là duy nhất
print(f"Number of unique compound words: {len(compound_words)}")
print(compound_words[:10])  # In ra 10 từ ghép đầu tiên để kiểm tra

Number of unique compound words: 14222
['moving_target', 'rung_giật', 'ruột_phân_su', 'chồng_lấn', 's_constellatus', 'sơn_phủ', 'bầu_dục', 'bong_bóng', 'đau_răng', 'akahata_y']


In [15]:
num_added_tokens = tokenizer.add_tokens(compound_words)

model.resize_token_embeddings(len(tokenizer))

print(f"Added {num_added_tokens} tokens. New vocab size: {len(tokenizer)}")

Added 14222 tokens. New vocab size: 54252


In [16]:
test_sentence = "rào_cản tự_nhiên chống lại nhiễm_trùng da"
tokens = tokenizer.tokenize(test_sentence)
print(tokens)

['rào_cản', 'tự_nhiên', '▁chống', '▁lại', 'nhiễm_trùng', '▁da']


### a. Khởi Tạo Keras Tokenizer

In [17]:
# # Define the Keras tokenizer with punctuation filters
# preprocessing_tokenizer = tf.keras.preprocessing.text.Tokenizer(
#     filters='!“"”#$%&()*+,-./:;<=>?@[\]^`{|}~ '
# )

In [18]:
def tokenize_data(titles, contents, tokenizer, title_max_length, content_max_length):
    def process_in_batches(texts, tokenizer, batch_size, max_length):
        encodings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]

            processed_texts = batch_texts

            # Đảm bảo tokenization đúng cách, giữ các từ đầy đủ
            batch_encodings = tokenizer(
                processed_texts,
                truncation=True,  # Cắt bớt các từ vượt quá max_length nếu cần
                padding='max_length',  # Thêm padding để đạt kích thước max_length
                max_length=max_length,
                return_tensors='pt'  # Trả về tensors để dễ dàng sử dụng trong PyTorch
            )
            encodings.append(batch_encodings)
        # Kết hợp tất cả encodings từ các batch
        return {key: torch.cat([batch[key] for batch in encodings], dim=0) for key in encodings[0]}

    # Tokenize titles and contents
    input_encodings = process_in_batches(titles, tokenizer, batch_size=32, max_length=title_max_length)
    answer_encodings = process_in_batches(contents, tokenizer, batch_size=32, max_length=content_max_length)

    return input_encodings, answer_encodings

# Tokenize the data
title_max_length = 76
content_max_length = 1024
input_encodings, answer_encodings = tokenize_data(titles, detailed_contents, tokenizer, title_max_length, content_max_length)

In [19]:
# Tính độ dài của mỗi câu trong input_encodings dựa trên attention_mask
input_lengths = input_encodings['attention_mask'].sum(dim=1)
input_max_length = input_lengths.max().item()

# Tính độ dài của mỗi câu trong answer_encodings dựa trên attention_mask
answer_lengths = answer_encodings['attention_mask'].sum(dim=1)
answer_max_length = answer_lengths.max().item()

print(f"Đoạn dài nhất trong input_encodings sau khi loại bỏ <pad> có kích thước {input_max_length} token.")
print(f"Đoạn dài nhất trong answer_encodings sau khi loại bỏ <pad> có kích thước {answer_max_length} token.")

Đoạn dài nhất trong input_encodings sau khi loại bỏ <pad> có kích thước 76 token.
Đoạn dài nhất trong answer_encodings sau khi loại bỏ <pad> có kích thước 1024 token.


In [20]:
# Hàm để kiểm tra tokenization
def check_tokenization(texts, encodings, tokenizer):
    for i in range(min(3, len(texts))):  # Chỉ in ra 3 ví dụ đầu tiên
        original_text = texts[i]
        token_ids = encodings['input_ids'][i]
        tokens = tokenizer.convert_ids_to_tokens(token_ids)
        decoded_text = tokenizer.decode(token_ids, skip_special_tokens=True)

        print(f"Original Text: {original_text}")
        print(f"Tokens: {tokens}")
        print(f"Decoded Text: {decoded_text}")
        print("="*50)

# Kiểm tra tokenization cho titles và contents
check_tokenization(titles, input_encodings, tokenizer)
check_tokenization(detailed_contents, answer_encodings, tokenizer)

Original Text: rào_cản tự_nhiên chống lại nhiễm_trùng da
Tokens: ['<s>', 'rào_cản', 'tự_nhiên', '▁chống', '▁lại', 'nhiễm_trùng', '▁da', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Decoded Text: rào_cản tự_nhiên chống lại nhiễm_trùng da
Original Text: rào_cản tự_nhiên chống lại nhiễm_trùng niêm_mạc
Tokens: ['<s>', 'rào_cản', 'tự_nhiên', '▁chống', '▁lại', 'nhiễm_trùng', 'niêm_mạc', '</s>', '<pad>', '<pad>', '<pad>', '<pad>

## Check word embeding

In [21]:
# Define a function to check word embeddings
def check_word_embeddings(texts, tokenizer):
    missing_words = set()
    all_words = set()

    # Tokenize each text and collect words
    for text in texts:
        # Ensure text is a string before tokenization
        if not isinstance(text, str):
            text = str(text)  # Convert to string if not already
        tokens = tokenizer.tokenize(text)
        all_words.update(tokens)

    # Check which words are in the tokenizer's vocabulary
    tokenizer_vocab = tokenizer.get_vocab()
    missing_words = [word for word in all_words if word not in tokenizer_vocab]

    return missing_words

In [22]:
# Check missing words
missing_words = check_word_embeddings(detailed_contents, tokenizer)

# Print results
print("Number of missing words:", len(missing_words))
print("Missing words:", missing_words)

Number of missing words: 1486
Missing words: ['▁expose', '▁jep', '▁decline', '▁menos', '▁puesto', '▁madh', '▁koske', '▁specifi', '▁fetish', '▁rull', 'gardiner', '▁dzi', '▁suna', '▁interpretation', '▁circula', '▁dado', '▁dono', '▁ramas', '▁renova', '▁boek', '▁wegen', '▁providing', '▁brett', '▁reag', '▁dvor', '▁hoge', '▁concurrent', '▁hev', '▁fato', '▁spontane', '▁circumstances', 'adequa', '▁dø', '▁efficient', '▁brann', '▁milito', '▁sebe', '▁preserva', '▁foram', '▁halos', '▁contribute', 'uose', '▁sensitive', '_', '▁examine', '▁limitation', '▁qe', '▁preventiv', '▁peut', '▁spiritual', '▁volunteer', '▁padu', '▁redistribu', 'eink', '▁siad', '▁guidelines', '▁besser', '▁campagna', '▁piedra', '▁recipient', '▁editorial', '▁buni', '▁sza', '▁kena', '▁xist', '▁norme', '▁individuals', '▁erro', '▁cerebro', '▁compara', '▁fiducia', '▁excita', '▁berge', '▁respirator', '▁advers', '▁puo', '▁quantitat', '▁epilepsi', '▁arian', '▁subir', '▁dominant', '▁benda', '▁peynir', '▁viser', '▁certa', '▁dialogue', '▁pr

## 5. Defining the Medical Dataset Class

### N-GRAM

In [23]:
class MedicalDataset(Dataset):
    def __init__(self, input_encodings, answer_encodings):
        self.input_encodings = input_encodings
        self.answer_encodings = answer_encodings

    def __len__(self):
        return len(self.input_encodings['input_ids'])

    def __getitem__(self, idx):
        input_ids = self.input_encodings['input_ids'][idx]
        attention_mask = self.input_encodings['attention_mask'][idx]
        labels = self.answer_encodings['input_ids'][idx]

        # Padding or truncating labels to match input_ids length
        if len(labels) < len(input_ids):
            labels = labels + [0] * (len(input_ids) - len(labels))  # Pad with 0
        elif len(labels) > len(input_ids):
            labels = labels[:len(input_ids)]  # Truncate labels

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [24]:
# Split data into train and validation sets
train_size = int(0.8 * len(input_encodings['input_ids']))
val_size = len(input_encodings['input_ids']) - train_size



train_dataset = MedicalDataset(
    {key: val[:train_size] for key, val in input_encodings.items()},
    {key: val[:train_size] for key, val in answer_encodings.items()}
)

val_dataset = MedicalDataset(
    {key: val[train_size:] for key, val in input_encodings.items()},
    {key: val[train_size:] for key, val in answer_encodings.items()}
)

### Dataloaders

In [25]:

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## **10. Training the Model**

In [26]:
def format_time(seconds):
    """
    Format time given in seconds into a string of format hh:mm:ss.

    Args:
        seconds (float): Time in seconds.

    Returns:
        str: Formatted time string.
    """
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

In [27]:
import time

def train(model, train_loader, dev_loader, optimizer, epochs, ignored_index, device='cuda', model_file='model'):
    """
    Train the model and evaluate it on the development set.

    Args:
        model: The model to train.
        train_loader: DataLoader for training data.
        dev_loader: DataLoader for development data.
        optimizer: Optimizer for training.
        epochs (int): Number of epochs to train.
        ignored_index (int): Index to ignore in the loss calculation.
        device (str): Device to run the model on ('cpu' or 'cuda'). Defaults to 'cuda'.
        model_file (str): File name to save the model checkpoints.

    Returns:
        train_epochs_loss: List of training losses for each epoch.
        dev_epochs_loss: List of development losses for each epoch.
        train_steps_loss: List of training losses for each step.
    """
    train_epochs_loss, dev_epochs_loss = [], []
    train_steps_loss = []

    lr = optimizer.param_groups[0]['lr']
    total_t0 = time.time()

    print("\n============================ Initial Evaluation ===========================\n")

    initial_dev_loss, preds = evaluate(model, dev_loader, device, ignored_index)
    dev_epochs_loss.append(initial_dev_loss)

    print(f"   Initial Dev Loss: {initial_dev_loss:.3f}\n")
    evaluation_time = format_time(time.time() - total_t0)
    print(f"   Initial Evaluation took: {evaluation_time}\n")

    for epoch in range(epochs):
        print(f"\n=============================== Epoch {epoch + 1} / {epochs} ===============================\n")
        print(" Training...\n")

        t0 = time.time()
        model.train()
        total_train_loss = 0.0
        sum_loss = 0.0

        for batch_idx, batch in enumerate(train_loader):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_train_loss += loss.item()
            sum_loss += loss.item()

            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % 1000 == 0:
                current = sum_loss / 1000
                train_steps_loss.append(current)
                elapsed = format_time(time.time() - t0)
                print(f"   Batch: {batch_idx + 1:>6,} / {len(train_loader):>5,}... Elapsed: {elapsed}... Train Loss: {current:.3f}...")
                sum_loss = 0.0

        avg_train_loss = total_train_loss / len(train_loader)
        train_epochs_loss.append(avg_train_loss)

        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, f"{model_file}-epoch{epoch + 1}.pt")

        print(f"\n   Average training loss: {avg_train_loss:.3f}")
        print(f"   Training epoch took: {format_time(time.time() - t0)}")

        print("\n Evaluating...\n")

        t1 = time.time()
        model.eval()
        avg_dev_loss, preds = evaluate(model, dev_loader, device, ignored_index)
        dev_epochs_loss.append(avg_dev_loss)

        print(f"   Dev Loss: {avg_dev_loss:.3f}\n")
        print("   Dev Score:")
        produce_predictions_file(preds, epoch + 1)

        evaluation_time = format_time(time.time() - t1)
        print(f"   Evaluation took: {evaluation_time}\n")

        epoch_time = format_time(time.time() - t0)
        print(f" Overall, Epoch took: {epoch_time}")

        if (epoch + 1) % 2 == 0:
            lr /= 2
            for group in optimizer.param_groups:
                group['lr'] = lr
            print(f"\n The learning rate updated to {group['lr']}")

    print(f"\nTraining completed!")
    print(f"Total training took {format_time(time.time() - total_t0)} (hh:mm:ss)")

    return train_epochs_loss, dev_epochs_loss, train_steps_loss

In [28]:
def evaluate(model, loader, device, ignored_index):
    model.eval()
    total_dev_loss = 0.0
    answers = []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # labels dùng cho BART để tính toán tổn thất

            # Forward pass.
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_dev_loss += loss.item()

            # Get predictions from model
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            # Convert predictions to strings
            for input_id, prediction in zip(input_ids, predictions):
                decoded_prediction = tokenizer.decode(prediction, skip_special_tokens=True)
                answers.append(decoded_prediction)

    avg_loss = total_dev_loss / len(loader)
    return avg_loss, answers

In [29]:
def produce_predictions_file(predictions, epoch):
    # Thực hiện lưu dự đoán vào tệp (hoặc xử lý khác nếu cần)
    with open(f'predictions_epoch_{epoch}.txt', 'w') as f:
        for pred in predictions:
            f.write(pred + '\n')

## 11. Start Training

In [30]:
ignored_index = -100  # thường dùng để bỏ qua các vị trí không quan trọng trong loss computation

train_epochs_loss, dev_epochs_loss, train_steps_loss = train(
    model,
    train_loader,
    val_loader,
    optimizer,
    epochs=10,
    ignored_index=ignored_index,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)


============================ Initial Evaluation ===========================



<ipython-input-23-be73958c2d03>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(input_ids, dtype=torch.long),
<ipython-input-23-be73958c2d03>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
<ipython-input-23-be73958c2d03>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(labels, dtype=torch.long)


   Initial Dev Loss: 15.645

   Initial Evaluation took: 00:00:54


=============================== Epoch 1 / 10 ===============================

 Training...


   Average training loss: 6.816
   Training epoch took: 00:12:14

 Evaluating...

   Dev Loss: 6.591

   Dev Score:
   Evaluation took: 00:00:56

 Overall, Epoch took: 00:13:11

=============================== Epoch 2 / 10 ===============================

 Training...



KeyboardInterrupt: 

## 12.Load the Fine-Tuned Model

In [ ]:
# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/ChatBox/model/fine_tuned_bartdemo1')
tokenizer.save_pretrained('/content/drive/MyDrive/ChatBox/model/fine_tuned_bartdemo1')

In [ ]:

# # Plot the performance
# plot_performance(batch_losses, epoch_losses)

In [ ]:
def generate_response(input_text: str, model: BartForConditionalGeneration, tokenizer: BartphoTokenizer, device: str = 'cpu', max_length: int = 1024, answer_max_length: int = 1024) -> str:
    """
    Generate a response for the input text using a pre-trained model and tokenizer.

    Args:
        input_text (str): Input text for generating the response.
        model (BartForConditionalGeneration): Pre-trained BART model.
        tokenizer (BartTokenizer): Tokenizer for BART.
        device (str): Device to run the model on ('cpu' or 'cuda'). Defaults to 'cpu'.
        max_length (int): Maximum length of the input text. Defaults to 1024.
        answer_max_length (int): Maximum length of the generated response. Defaults to 1024.

    Returns:
        str: The generated response by the Chatbot.
    """
    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt',
                                  truncation=True, padding='max_length', max_length=max_length)
    input_ids = input_ids.to(device)

    # Move the model to the device
    model.to(device)  # Ensure the model is on the same device

    # Generate response using the model
    with torch.no_grad():
        output = model.generate(input_ids,
                               max_length=answer_max_length,
                               do_sample=True,
                               top_k=50,  # Use top-k sampling
                               top_p=0.92,  # Use top-p sampling
                               temperature=0.7,
                               no_repeat_ngram_size=1,
                               early_stopping=True)

    # Decode the generated response
    response = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return response

## 13. Example: Generating a Response to a User's Question

In [ ]:
# Đặt câu hỏi của người dùng
user_question = "tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịch áp_lực tĩnh_mạch trung_tâm"

# Định nghĩa các tham số cần thiết
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Gọi hàm generate_response1 với các đối số cần thiết
response = generate_response(
    input_text=user_question,
    model=model,  # Đây là mô hình đã được tải và di chuyển đến thiết bị
    tokenizer=tokenizer,  # Đây là tokenizer đã được tải
    device=device,
    max_length=72,
    answer_max_length=answer_max_length
)

# In ra phản hồi đã được sinh ra
print(f"Generated Response: {response}")

In [ ]:
# # Example user question
# user_question = "dấu hiệu của nhiễm trùng"

# # Generate response
# responses = generate_response(user_question, model, tokenizer, titles, detailed_contents)

# print(responses)

# # # Print generated responses
# # for idx, (similar_question, corresponding_answer, refined_answers) in enumerate(responses):
# #     print(f"\nResponse {idx + 1}:")
# #     print(f"Similar Question: {similar_question}")
# #     print(f"Original Answer: {corresponding_answer}")
# #     print(f"Refined Answer: {refined_answers}")

#Use the Fine-Tuned Model in the Chatbox